# Task 6 - RSA
Krypto Lab

Felix Kleinsteuber $\cdot$ Matrikelnr.: 185709

In [1]:
import numpy as np

## 1. Quadrieren und Multiplizieren
Wir quadrieren die Basis wiederholt und multiplizieren die Quadrate auf, um effizienter modulo $n$ potenzieren zu können. Das Prinzip basiert auf den Potenz- und Modulogesetzen, z.B.:

\begin{align*}
18^7 \mod 3 &= 18^1 \cdot 18^2 \cdot 18^4 \mod 3 \\
&= (18 \mod 3) \cdot ((18 \mod 3)^2 \mod 3) \cdot (((18 \mod 3)^2 \mod 3)^2 \mod 3)
\end{align*}

In [2]:
def quad_and_mult(x, m, n):
    """Berechnet effizient x^m mod n."""
    y = 1
    while m != 0:
        if m & 0x1 != 0:
            # falls bit=1: Multipliziere y mit x
            y = (y * x) % n
        # für jedes Bit wird x quadriert
        x = (x * x) % n
        # schiebe zum nächsten Bit
        m >>= 1
    return y

# random tests
assert quad_and_mult(2, 5, 10) == (2 ** 5) % 10
assert quad_and_mult(41, 13, 33) == (41 ** 13) % 33

## 2. Erweiterter Euklidischer Algorithmus (eeA)
Wir nutzen diesen, um für bekanntes $\phi(n) = (p - 1) (q - 1)$ ein $d$ zu gegebenem $e$ zu berechnen, sodass gilt: $de \equiv 1 \mod \phi(n)$.

In [3]:
def extended_euclidian(a, b):
    """ Berechnet ggT(a,b) = sa + tb = r und gibt r, s, t zurück. """
    r = [a, b]
    s = [1, 0]
    t = [0, 1]
    while r[-1] != 0:
        q = r[-2] // r[-1]
        r.append(r[-2] - q * r[-1])
        s.append(s[-2] - q * s[-1])
        t.append(t[-2] - q * t[-1])
    return r[-2], s[-2], t[-2]

# d berechnen
p, q = 11, 7
n = p * q
e = 53
phi_n = (p - 1) * (q - 1)
ggT, _, d = extended_euclidian(phi_n, e)
assert ggT == 1
assert (d * e) % phi_n == 1
print(f"d: {d}, e: {e}, p: {p}, q: {q}, n: {n}, phi(n): {phi_n}")

d: 17, e: 53, p: 11, q: 7, n: 77, phi(n): 60


## 3. RSA Encrypt / Decrypt
Die eigentlichen Encrypt/Decrypt-Funktionen:
* $E(e, n) = x^e \mod n$
* $D(d, n) = x^d \mod n$

Wir nutzen dafür die effiziente Implementierung (Quadrieren und Multiplizieren).

In [4]:
def rsa_encrypt(e, n, x):
    assert np.all(x < n)
    return np.array([quad_and_mult(x1, e, n) for x1 in x])

def rsa_decrypt(d, n, y):
    # äquivalent zu rsa_encrypt, aber der Übersichtlichkeit halber mit anderen Variablennamen
    assert np.all(y < n)
    return np.array([quad_and_mult(y1, d, n) for y1 in y])

x_sample = np.random.randint(0, n, 5000)
print("sample input:", x_sample)
%time enc = rsa_encrypt(e, n, x_sample)
print("encrypted:", enc)
%time dec = rsa_decrypt(d, n, enc)
print("decrypted", dec)
assert np.array_equal(x_sample, dec)

sample input: [36 25 21 ... 70 72  9]
Wall time: 49 ms
encrypted: [71 16 21 ... 42 18 25]
Wall time: 26 ms
decrypted [36 25 21 ... 70 72  9]


Encrypt und Decrypt sind wegen des sehr kleinen $n$ s sehr schnell. Encrypt und Decrypt hintereinander erzeugen - wie erhofft - wieder den Klartext.